In [1]:
import os
from pyspark.sql import Row
from datetime import datetime
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.sql.window import Window
from pyspark.storagelevel import StorageLevel
import pyspark.sql.functions as f
from pyspark import SparkContext
from pyspark.conf import SparkConf

In [2]:
pyspark_submit_args = '--packages org.mongodb.spark:mongo-spark-connector_2.11:2.4.0 pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

spark = SparkSession \
    .builder \
    .appName("aqi") \
    .config("spark.mongodb.input.uri", "mongodb://34.221.229.103/mydb.air")\
    .getOrCreate()
spark.conf.set("spark.executor.memory", '4g')
spark.conf.set('spark.executor.cores', '4')
spark.conf.set('spark.cores.max', '4')
spark.conf.set("spark.driver.memory",'4g')
df = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()
df.printSchema()

root
 |-- Unnamed: 0.1.1: integer (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- aqi_co: double (nullable = true)
 |-- aqi_no2: double (nullable = true)
 |-- aqi_o3: double (nullable = true)
 |-- aqi_pm10: double (nullable = true)
 |-- aqi_pm25_frm: double (nullable = true)
 |-- aqi_pm25_nonfrm: double (nullable = true)
 |-- aqi_so2: double (nullable = true)
 |-- arithmetic_mean_co: double (nullable = true)
 |-- arithmetic_mean_no2: double (nullable = true)
 |-- arithmetic_mean_o3: double (nullable = true)
 |-- arithmetic_mean_pm10: double (nullable = true)
 |-- arithmetic_mean_pm25_frm: double (nullable = true)
 |-- arithmetic_mean_pm25_nonfrm: double (nullable = true)
 |-- arithmetic_mean_pm25_speciation: double (nullable = true)
 |-- arithmetic_mean_pressure: double (nullable = true)
 |-- arithmetic_mean_so2: double (nullable = true)
 |-- arithmetic_mean_temp: double (nullable = true)
 |-- arithmetic_mean_wind: double (nullable = tr

In [4]:
va = VectorAssembler(outputCol="features", inputCols=[
 'aqi_co',
 'aqi_no2',
 'aqi_o3',
 'aqi_pm10',
 'aqi_pm25_frm',
 'aqi_pm25_nonfrm',
 'aqi_so2',
 'arithmetic_mean_co',
 'arithmetic_mean_no2',
 'arithmetic_mean_o3',
 'arithmetic_mean_pm10',
 'arithmetic_mean_pm25_frm',
 'arithmetic_mean_pm25_nonfrm',
 'arithmetic_mean_pm25_speciation',
 'arithmetic_mean_pressure',
 'arithmetic_mean_so2',
 'arithmetic_mean_temp',
 'arithmetic_mean_wind',
 'county_code',
 'day',
 'dow',
 'first_max_value_co',
 'first_max_value_no2',
 'first_max_value_o3',
 'first_max_value_pm10',
 'first_max_value_pm25_frm',
 'first_max_value_pm25_nonfrm',
 'first_max_value_pm25_speciation',
 'first_max_value_pressure',
 'first_max_value_so2',
 'first_max_value_temp',
 'first_max_value_wind',
 'latitude',
 'longitude',
 'max_aqi',
 'max_aqi_before_yesterday',
 'max_aqi_yesterday',
 'month',
 'observation_count_co',
 'observation_count_no2',
 'observation_count_o3',
 'observation_count_pm10',
 'observation_count_pm25_frm',
 'observation_count_pm25_nonfrm',
 'observation_count_pm25_speciation',
 'observation_count_pressure',
 'observation_count_so2',
 'observation_count_temp',
 'observation_count_wind',
 'site_num']) 
df2 = va.transform(df).select("features", "label")
df2.show()

+--------------------+--------+
|            features|   label|
+--------------------+--------+
|[5.0,18.0,14.0,19...|    good|
|[5.0,18.0,31.0,19...|    good|
|[5.0,18.0,27.0,19...|    good|
|[5.0,18.0,33.0,19...|    good|
|[5.0,18.0,33.0,19...|    good|
|[5.0,18.0,36.0,19...|    good|
|[5.0,18.0,37.0,19...|    good|
|[5.0,18.0,37.0,19...|    good|
|[5.0,18.0,28.0,19...|    good|
|[5.0,18.0,27.0,19...|    good|
|[5.0,18.0,34.0,19...|    good|
|[5.0,18.0,46.0,19...|moderate|
|[5.0,18.0,77.0,19...|    good|
|[5.0,18.0,74.0,19...|moderate|
|[5.0,18.0,40.0,19...|    good|
|[5.0,18.0,31.0,19...|    good|
|[5.0,18.0,38.0,19...|    good|
|[5.0,18.0,42.0,19...|    good|
|[5.0,18.0,43.0,19...|    good|
|[5.0,18.0,39.0,19...|    good|
+--------------------+--------+
only showing top 20 rows



In [8]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load and parse the data file, converting it to a DataFrame.
data = df2

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
#featureIndexer =\
#    VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [9]:
%%time
# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="features", numTrees=10)

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, rf, labelConverter])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("predictedLabel", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator =  MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))
rfModel = model.stages[1]
print(rfModel)  # summary only

+--------------+-----+--------------------+
|predictedLabel|label|            features|
+--------------+-----+--------------------+
|          good| good|[0.0,8.0,14.0,19....|
|          good| good|[1.0,7.0,25.0,19....|
|          good| good|[1.0,8.0,28.0,11....|
|          good| good|[1.0,8.0,33.0,19....|
|          good| good|[1.0,9.0,19.0,9.0...|
+--------------+-----+--------------------+
only showing top 5 rows

Test Error = 0.190449
RandomForestClassificationModel (uid=RandomForestClassifier_4b07af5f740288ac3b66) with 10 trees
CPU times: user 296 ms, sys: 229 ms, total: 525 ms
Wall time: 40min 57s


In [ ]:
lr = LogisticRegression(labelCol="indexedLabel", featuresCol="features",maxIter=10, regParam=0.3, elasticNetParam=0.8)

pipeline = Pipeline(stages=[labelIndexer, rf, labelConverter])
# Fit the model
model2 = pipeline.fit(trainingData)
predictions2 = model2.transform(testData)
predictions2.select("predictedLabel", "label", "features").show(5)
# Print the coefficients and intercept for multinomial logistic regression
print("Coefficients: \n" + str(model2.coefficientMatrix))
print("Intercept: " + str(model2.interceptVector))

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Train a GBT model.
gbt = GBTClassifier(labelCol="indexedLabel", featuresCol="features", maxIter=10)

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("predictedLabel", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

gbtModel = model.stages[2]
print(gbtModel)  # summary only

In [ ]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="f1")
f1 = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="precision")
precision = evaluator.evaluate(predictions)

evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="recall")
recall = evaluator.evaluate(predictions)